In [1]:
import ee
import geemap

Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [2]:
import os
os.chdir(r'D:\Work\naqu')

In [3]:
shp_file = r'naqu.shp'
shp = geemap.shp_to_ee(shp_file)

Map.addLayer(shp, {}, 'shp')

Map.center_object(shp, 3)
roi = shp.geometry()

In [4]:
start_time = 2018
end_time = 2021
day_sep = 8
bands = ['temperature_2m',
        'dewpoint_temperature_2m', 
        'total_precipitation', 
        'surface_net_thermal_radiation',
        'surface_net_solar_radiation',
        'surface_solar_radiation_downwards',
        'u_component_of_wind_10m',
        'v_component_of_wind_10m']

In [5]:
Albers_project_img = ee.Image('users/2431566134liumonarch/WGS_1984_Albers_project')

crs = Albers_project_img.projection()

In [6]:
bands[6]

'u_component_of_wind_10m'

In [11]:
year = 2018
day = 0
images_year = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{year}', f'{year+1}')
images_day = images_year.filter(ee.Filter.calendarRange(day, day+30, 'day_of_year'))
# img_day = images_day.select(bands[6:8]).mean()
# Map.addLayer(img_day.clip(roi), {}, 'img')

In [12]:
img_day = images_year.mean().clip(roi).select(bands[6:8], ['u', 'v'])

In [13]:
img = img_day.expression("(u ** 2 + v ** 2) ** 0.5", 
                         {
                             'u': img_day.select('u'),
                             'v': img_day.select('v')
                         })

In [ ]:
geemap.download_ee_image(img, 'wind_250.tif', region=roi, crs=crs, scale=250)

wind_250.tif: |                                                       | 0.00/91.6M (raw) [  0.0%] in 00:00 (et…

In [ ]:
img_day = images_day.select(bands[2]).sum()
Map.addLayer(img_day, {}, 'prcp_s')

#### PRCP 8day download

In [ ]:
from Monarch.user_gee import *

In [ ]:
var = 'PRCP'
if not os.path.exists(var):
    os.mkdir(var)
    
for year in range(start_time, end_time):
    if (year%4 == 0 and year%400!=0) or (year%400==0):
        days = 366
    else:
        days = 365
    images_year = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{year}', f'{year+1}')
    
    imgs_day_to_year = []
    for day in range(days):
        images_day = images_year.filter(ee.Filter.calendarRange(day, day+1, 'day_of_year'))

        img_day = images_day.select(bands[2]).sum()

        imgs_day_to_year.append(img_day)

    for i in range(0, days, day_sep):
        image = ee.ImageCollection(imgs_day_to_year[i:i+day_sep]).sum().multiply(1000)
        outfile = f'{var}/{var}_{year}{i+1:03d}.tif'
        while not os.path.exists(outfile):
            try:
                download_image(image, outfile, region=roi, crs=crs,
                                     scale=10000)
            except:
                continue

#### TAVG 8day download

In [ ]:
var = 'TAVG'
if not os.path.exists(var):
    os.mkdir(var)
    
for year in range(start_time, end_time):
    if (year%4 == 0 and year%400!=0) or (year%400==0):
        days = 366
    else:
        days = 365
    images_year = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{year}', f'{year+1}')
    
    imgs_day_to_year = []
    for day in range(days):
        images_day = images_year.filter(ee.Filter.calendarRange(day, day+1, 'day_of_year'))

        img_day = images_day.select(bands[0]).mean()

        imgs_day_to_year.append(img_day)

    for i in range(0, days, day_sep):
        image = ee.ImageCollection(imgs_day_to_year[i:i+day_sep]).mean().subtract(273.15)
        outfile = f'{var}/{var}_{year}{i+1:03d}.tif'
        while not os.path.exists(outfile):
            try:
                download_image(image, outfile, region=roi, crs=crs,
                                     scale=10000)
            except:
                continue
            

#### TMIN 8day download

In [ ]:
var = 'TMIN'
if not os.path.exists(var):
    os.mkdir(var)
    
for year in range(start_time, end_time):
    if (year%4 == 0 and year%400!=0) or (year%400==0):
        days = 366
    else:
        days = 365
    images_year = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{year}', f'{year+1}')
    
    imgs_day_to_year = []
    for day in range(days):
        images_day = images_year.filter(ee.Filter.calendarRange(day, day+1, 'day_of_year'))

        img_day = images_day.select(bands[0]).min()

        imgs_day_to_year.append(img_day)

    for i in range(0, days, day_sep):
        image = ee.ImageCollection(imgs_day_to_year[i:i+day_sep]).mean().subtract(273.15)
        outfile = f'{var}/{var}_{year}{i+1:03d}.tif'
        while not os.path.exists(outfile):
            try:
                download_image(image, outfile, region=roi, crs=crs,
                                     scale=10000)
            except:
                continue

#### TMAX 8day download

In [ ]:
def calc_RHU(img):
    Ea = img.expression('0.6108 * exp(17.27 * b(3) / (b(3) + 237.3))')
    Etmax = img.expression('0.6108 * exp(17.27 * b(1) / (b(1) + 237.3))')
    Etmin = img.expression('0.6108 * exp(17.27 * b(0) / (b(0) + 237.3))')
    return Ea.divide(Etmax.add(Etmin).divide(2)).multiply(100)

In [ ]:
var = 'TMAX'
if not os.path.exists(var):
    os.mkdir(var)
    
for year in range(start_time, end_time):
    if (year%4 == 0 and year%400!=0) or (year%400==0):
        days = 366
    else:
        days = 365
    images_year = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{year}', f'{year+1}')
    
    imgs_day_to_year = []
    for day in range(days):
        images_day = images_year.filter(ee.Filter.calendarRange(day, day+1, 'day_of_year'))

        img_day = images_day.select(bands[0]).max()

        imgs_day_to_year.append(img_day)

    for i in range(0, days, day_sep):
        image = ee.ImageCollection(imgs_day_to_year[i:i+day_sep]).mean().subtract(273.15)
        outfile = f'{var}/{var}_{year}{i+1:03d}.tif'
        while not os.path.exists(outfile):
            try:
                download_image(image, outfile, region=roi, crs=crs,
                                     scale=10000)
            except:
                continue

#### WIND 8day download

In [ ]:
var = 'WIND'
if not os.path.exists(var):
    os.mkdir(var)
    
for year in range(start_time, end_time):
    if (year%4 == 0 and year%400!=0) or (year%400==0):
        days = 366
    else:
        days = 365
    images_year = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{year}', f'{year+1}')
    
    imgs_day_to_year = []
    for day in range(days):
        images_day = images_year.filter(ee.Filter.calendarRange(day, day+1, 'day_of_year'))

        img_day = images_day.select(bands[6:8])\
                .map(lambda img: img.expression('sqrt(b(0)**2+b(1)**2)')).mean()

        imgs_day_to_year.append(img_day)

    for i in range(0, days, day_sep):
        image = ee.ImageCollection(imgs_day_to_year[i:i+day_sep]).mean()
        outfile = f'{var}/{var}_{year}{i+1:03d}.tif'
        while not os.path.exists(outfile):
            try:
                download_image(image, outfile, region=roi, crs=crs,
                                     scale=10000)
            except:
                continue

#### RHU 8day download

In [ ]:
var = 'RHU'
if not os.path.exists(var):
    os.mkdir(var)
    
for year in range(start_time, end_time):
    if (year%4 == 0 and year%400!=0) or (year%400==0):
        days = 366
    else:
        days = 365
    images_year = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{year}', f'{year+1}')
    
    imgs_day_to_year = []
    for day in range(days):
        images_day = images_year.filter(ee.Filter.calendarRange(day, day+1, 'day_of_year'))

        tmax = images_day.select(bands[0]).max()
        tmin = images_day.select(bands[0]).min()
        tdew = images_day.select(bands[1]).mean()
        img = ee.Image([tmax, tmin, tdew])
        Ea = img.expression('0.6108 * exp(17.27 * b(2) / (237.3 + b(2)))')
        Etmax = img.expression('0.6108 * exp(17.27 * b(0) / (237.3 + b(0)))')
        Etmin = img.expression('0.6108 * exp(17.27 * b(1) / (237.3 + b(1)))')
        Es = Etmax.add(Etmin).divide(2)
        img_day = Ea.divide(Es).multiply(100)
        
        imgs_day_to_year.append(img_day)

    for i in range(0, days, day_sep):
        image = ee.ImageCollection(imgs_day_to_year[i:i+day_sep]).mean()
        outfile = f'{var}/{var}_{year}{i+1:03d}.tif'
        while not os.path.exists(outfile):
            try:
                download_image(image, outfile, region=roi, crs=crs,
                                     scale=10000)
            except:
                continue

#### SNR 8day download

In [ ]:
var = 'SNR'
if not os.path.exists(var):
    os.mkdir(var)
    
for year in range(start_time, end_time):
    if (year%4 == 0 and year%400!=0) or (year%400==0):
        days = 366
    else:
        days = 365
    images_year = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{year}', f'{year+1}')
    
    imgs_day_to_year = []
    for day in range(days):
        images_day = images_year.filter(ee.Filter.calendarRange(day, day+1, 'day_of_year'))

        sntr = images_day.select(bands[3]).sum()
        snsr = images_day.select(bands[4]).sum()
        img_day = sntr.add(snsr)
        
        imgs_day_to_year.append(img_day)

    for i in range(0, days, day_sep):
        image = ee.ImageCollection(imgs_day_to_year[i:i+day_sep]).mean().divide(1000000)
        outfile = f'{var}/{var}_{year}{i+1:03d}.tif'
        while not os.path.exists(outfile):
            try:
                download_image(image, outfile, region=roi, crs=crs,
                                     scale=10000)
            except:
                continue


#### SSRD 8day download

In [ ]:
var = 'SSRD'
if not os.path.exists(var):
    os.mkdir(var)
    
for year in range(start_time, end_time):
    if (year%4 == 0 and year%400!=0) or (year%400==0):
        days = 366
    else:
        days = 365
    images_year = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")\
                .filterDate(f'{year}', f'{year+1}')
    
    imgs_day_to_year = []
    for day in range(days):
        images_day = images_year.filter(ee.Filter.calendarRange(day, day+1, 'day_of_year'))

        img_day = images_day.select(bands[5]).sum()
        
        imgs_day_to_year.append(img_day)

    for i in range(0, days, day_sep):
        image = ee.ImageCollection(imgs_day_to_year[i:i+day_sep]).mean().divide(1000000)
        outfile = f'{var}/{var}_{year}{i+1:03d}.tif'
        while not os.path.exists(outfile):
            try:
                download_image(image, outfile, region=roi, crs=crs,
                                     scale=10000)
            except:
                continue

In [ ]:
f'{var}/{var}_{year}{(i)+1:03d}.tif'

In [ ]:
for i in range(0, days, day_sep):
    print(i)